In [7]:
import wandb
import os

In [8]:
RUN_PATH = "team-epoch-iv/detect-harmful-brain-activity/seykjhek"

In [9]:
api = wandb.Api()
run = api.run(f"{RUN_PATH}")

In [10]:
is_sweep_run = run.sweep is not None
print(f"Is sweep run: {is_sweep_run}")

Is sweep run: False


In [11]:
# list all artefacts
model_artefacts = []
for artefact in run.logged_artifacts():
    if artefact.type == "model":
        print(artefact.name)
        model_artefacts.append(artefact)

EEGNet:v1782
EEGNet:v1785
EEGNet:v1787
EEGNet:v1789
EEGNet:v1792
EEGNet:v1794
EEGNet:v1797
EEGNet:v1799
EEGNet:v1801
EEGNet:v1803
EEGNet:v1805
EEGNet:v1807
EEGNet:v1810
EEGNet:v1811
EEGNet:v1816
EEGNet:v1817
EEGNet:v1818
EEGNet:v1819
EEGNet:v1820
EEGNet:v1821


In [12]:
# download all artefacts
for artefact in model_artefacts:
    for file in artefact.files():
        print(f"Downloading {file.name} from {artefact.name}")
        file.download(root="../downloads")

In [13]:
# get the config type artefact (should be only one)
config_artefact = None
for artefact in run.logged_artifacts():
    if artefact.type == "config":
        config_artefact = artefact
        break

In [14]:
if not is_sweep_run:
    # download the raw, human readable config file
    for file in config_artefact.files():
        if file.name not in ["config.yaml","cv.yaml","train.yaml"]:
            print(f"Downloading {file.name} from {config_artefact.name}")
            file.download(root="../downloads", replace=True)
            # rename the file to the name of the run
            os.rename(f"../downloads/{file.name}", f"../downloads/{run.name}")
            print(f"Renamed to {run.name}")

Renamed to regular-temple


In [15]:
from omegaconf import OmegaConf

if is_sweep_run:
    # download the file called config.yaml from the artifact, which includes sweep parameters
    for file in config_artefact.files():
        if file.name == "config.yaml":
            print(f"Downloading {file.name} from {config_artefact.name}")
            file.download(root="../downloads", replace=True)
            
            # read the file into a dictconfig with omegaconf
            config = OmegaConf.load(f"../downloads/{file.name}")        
            config = config.model
            # save the config again
            OmegaConf.save(config, f"../downloads/{file.name}")        
            
            # rename the file to the name of the run
            os.rename(f"../downloads/{file.name}", f"../downloads/{run.name}.yaml")
            print(f"Renamed to {run.name}.yaml")
